# Introducción a la Creación de Chatbots con LangChain y Modelos de Código Abierto

Por William Wong Garay - CTO de Azist.ai

### Herramientas que usaremos:

- [LangChain](https://python.langchain.com/): Framework para desarrollar aplicaciones poteniadas por grandes modelos de lenguaje natural (LLMs por sus siglas en inglés).
- [Groq](https://groq.com/):  Es una startup de Silicon Valley que provee servicios en la nube para acceder a LLMs OpenSource con respuestas de alta velocidad, gracias a sus chips LPU especializados. 
- Otras herramientas: Usaremos otras librerías como BeautifulSoup para la carga de documentos via web, y Chroma para la búsqueda y recuperación de información.

In [1]:
%pip install --upgrade pip -q
%pip install langchain -q
%pip install python-dotenv -q
%pip install langchain-groq -q
%pip install langchain-community -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Inicialización del LLM para el modelo de Chat

Para el ejemplo usaremos el modelo Llama 3.1 mediante la API de Groq.

Los modelos tipo Chat son modelos de lenguaje natural que han sido entrenados para recibir como entrada una pregunta de un user (HumanMessage) y devolver una respuesta de un agente virtual (AIMessage)

Pueden ver una lista completa de las librerías de chat disponibles en https://python.langchain.com/v0.1/docs/integrations/chat/

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv(override=True) # Carga el GROQ_API_KEY desde el archivo .env

True

In [7]:
llm = ChatGroq(
    temperature=1.0, # Entre 0 y 2. Temperaturas altas generan respuestas más creativas, temperaturas bajas generan respuestas más enfocadas y centradas.
    model="llama-3.1-8b-instant", # Modelo de lenguaje a utilizar. https://console.groq.com/docs/models
    # model="gemma2-9b-it",
    # model="llama3-70b-8192",
    verbose=True
)

llm.invoke("Hola, ¿conoces a la empresa Shalom?")


AIMessage(content='Buenos días. Shalom es una empresa de atención integral a personas mayores que opera en varios países de América Latina. La empresa ofrece servicios de residencias, hospitales y clínicas de atención al adulto mayor.\n\nShalom cuenta con un enfoque humanizado y de calidad en el cuidado de las personas mayores, ofreciendo servicios como cuidado médico, fonoaudiología, fisioterapia, asesoría nutricional, entre otros. La empresa también Prioriza el mantenimiento de la autonomía y el bienestar de los residentes.\n\nTambién tiene una larga historia y operaciones en estados de la región como los de Argentina así como en otros países anteriores a estos.\n\n¿Qué información específica sobre Shalom estás buscando? Estoy aquí para ayudarte.', response_metadata={'token_usage': {'completion_tokens': 172, 'prompt_tokens': 47, 'total_tokens': 219, 'completion_time': 0.229333333, 'prompt_time': 0.010138241, 'queue_time': 0.004156439000000001, 'total_time': 0.239471574}, 'model_name'

In [8]:
from langchain_core.messages import HumanMessage

llm.invoke(
    [ HumanMessage(content="Hola, conoces a la empresa Shalom en la ciudad de Piura?") ]
)

AIMessage(content='Lo siento, pero no tengo información específica sobre la empresa Shalom en la ciudad de Piura. La información disponible sobre empresas puede cambiarse rápidamente y la base de datos que tengo puede no estar actualizada al día.\n\nSin embargo, puedo sugerirte algunas opciones para que puedas encontrar más información sobre la empresa Shalom en Piura:\n\n1. Busca en buscadores: Puedes intentar búsqueda la información de la empresa en motores de búsqueda como Google, Bing, etc. También puedes usar palabras clave como "Shalom Piura" o "Shalom Perú" para obtener resultados más relevantes.\n2. Revisa directorios de empresas: Puedes revisar directorios de empresas en línea como Empresas Perú, Infoempleo, etc. para ver si la empresa Shalom está registrada en ellos.\n3. Verifica en redes sociales: Puedes buscar la empresa en redes sociales como Facebook, Instagram, Twitter, etc. para ver si tienen una presencia en línea.\n4. Contacta directamente: Puedes intentar contactar d

La conversacion con un Chat puede recibir principalmente 3 tipos de mensajes:

- SystemMessage: Mensajes del sistema, que corresponden al Prompt inicial del Chat. Sirve para darle contexto a la conversación.
- HumanMessage: Mensajes del usuario, que corresponden a las preguntas que el usuario le hace al Chat.
- AIMessage: Mensajes del Chat, que corresponden a las respuestas que el Chat le da al usuario a través del modelo de lenguaje natural.

In [9]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

res = llm.invoke([
    SystemMessage(content="Eres un asistente virtual. Tu nombre es Azzy y trabajas en Shalom. Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo"),
    HumanMessage(content="Hola, ¿como estas?"),
    AIMessage(content="Estoy bien, gracias por preguntar. En que te puedo ayudar?"),
    HumanMessage(content="Me das tu nombre? Cuentame mas sobre Shalom"),
])

print(res)


content='Me llamo Azzy y estoy a tu disposición para ayudarte en lo que necesites.\n\nY con gusto te informo que trabajo para Shalom, una empresa de courier especializada en envíos a través de diferentes destinos en el norte del Perú, específicamente a Piura, Lima y Chiclayo. Nuestra empresa se enfoca en brindar servicios de entrega rápida y confiable a nuestros clientes, tanto individuales como comerciales.\n\nNuestra misión es hacer que los envíos sean fáciles y seguros, permitiendo que nuestros clientes se concentren en lo que para ellos es más importante. ¿Necesitas realizar un envío?' response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 122, 'total_tokens': 266, 'completion_time': 0.192, 'prompt_time': 0.026669143, 'queue_time': 0.0026422670000000002, 'total_time': 0.218669143}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None} id='run-20b07ae4-df21-4b48-89ec-4a33634b0fc6-0' usage_metada

El Prompt es una parte importante del Chatbot, ya que es el mensaje inicial que se le da al modelo para que pueda responder de manera coherente.

LangChain nos pemite darle formato al Prompt mediante el uso de templates y variables, la cuales se pueden reemplazar por valores específicos en tiempo de ejecución.

Con el ChatPromptTemplate podremos pre formatear el Prompt del Chatbot y la cadena de mensajes que se le pasará al modelo por medio de MessagePlaceholder.

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente virtual. Tu nombre es {assistant_name} y trabajas en {company}. {description}."),
    MessagesPlaceholder(variable_name="messages"),
])

res = prompt.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es una empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": [
        HumanMessage(content="Hola, ¿como estas?"),
        AIMessage(content="Estoy bien, gracias por preguntar. En que te puedo ayudar?"),
        HumanMessage(content="Me das tu nombre? Hacen envios a Trujillo?"),
    ]
})

print(res)

messages=[SystemMessage(content='Eres un asistente virtual. Tu nombre es Azzy y trabajas en Shalom. Shalom es una empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo.'), HumanMessage(content='Hola, ¿como estas?'), AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'), HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?')]


Las cadenas en LangChain permiten combinar componentes del framework para encadenar las salidas de un componente a la entrada de otro.

En este ejemplo, usaremos el ChatPromptTemplate para formatear el Prompt del Chatbot y el MessagePlaceholder para formatear la cadena de mensajes que se le pasará al modelo ChatGroq.

Al hecer el invoke a la cadena, se le debe pasar todas las variables necesarias para que el Chatbot pueda responder de manera correcta.

In [11]:
chain = prompt | llm

chain.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": [
        HumanMessage(content="Hola, ¿como estas?"),
        AIMessage(content="Estoy bien, gracias por preguntar. En que te puedo ayudar?"),
        HumanMessage(content="Me das tu nombre? Hacen envios a Trujillo?"),
    ]
})

AIMessage(content='Mi nombre es Azzy, soy un asistente virtual de Shalom, una empresa de courier. Lo siento, pero Shalom únicamente ofrece envíos a las ciudades de Piura, Lima y Chiclayo. Trujillo no se encuentra dentro de nuestro alcance geográfico actual. ¿Necesitas hacer un envío a una de estas ciudades?', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 125, 'total_tokens': 207, 'completion_time': 0.109333333, 'prompt_time': 0.027371956, 'queue_time': 0.0024308440000000014, 'total_time': 0.136705289}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-6a835391-4d80-42c6-9341-083cf820afae-0', usage_metadata={'input_tokens': 125, 'output_tokens': 82, 'total_tokens': 207})

### Usando un Historial de Conversación

LangChain nos brinda diferentes librerías para trabajar con historiales de conversación, como ChatMessageHistory via langchain memory que permite almacenar en memoria los mensajes de la conversación.

Existen otras librerías que hacen uso de bases de datos permanentes para almacenar los mensajes de la conversación, como PostgresChatMessageHistory que permite almacenar los mensajes de la conversación en una base de datos Postgres.

Pueden ver una lista completa en https://python.langchain.com/v0.1/docs/integrations/memory/

In [12]:
from langchain.memory import ChatMessageHistory
chat_history = ChatMessageHistory()

chat_history.add_user_message("Hola, ¿como estas?")
chat_history.add_ai_message("Estoy bien, gracias por preguntar. En que te puedo ayudar?")
chat_history.add_user_message("Me das tu nombre? Hacen envios a Trujillo?")

chat_history.messages

[HumanMessage(content='Hola, ¿como estas?'),
 AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'),
 HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?')]

In [13]:
chat_history.add_ai_message("Soy Azzy, asistente virtual de Shalom. Solo hacemos envios a Piura, Lima y Chiclayo")
chat_history.add_user_message("Lástima, necesitaba un envio a Trujillo. Gracias por tu ayuda")

chat_history.messages

[HumanMessage(content='Hola, ¿como estas?'),
 AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'),
 HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?'),
 AIMessage(content='Soy Azzy, asistente virtual de Shalom. Solo hacemos envios a Piura, Lima y Chiclayo'),
 HumanMessage(content='Lástima, necesitaba un envio a Trujillo. Gracias por tu ayuda')]

In [14]:
ai_response = chain.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": chat_history.messages
})

ai_response

AIMessage(content='Lo siento, no podemos ayudarte con envíos a Trujillo. Si necesitas un servicio de courier a otras ciudades, tal vez podrías considerar otra empresa. Pero si estás planeando viajar a Piura, Lima o Chiclayo, ¡nosotros te podemos ayudar!', response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 182, 'total_tokens': 247, 'completion_time': 0.086666667, 'prompt_time': 0.049640882, 'queue_time': 0.0026035980000000056, 'total_time': 0.136307549}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-b346ab34-561d-42ed-bbc2-61e552a87a65-0', usage_metadata={'input_tokens': 182, 'output_tokens': 65, 'total_tokens': 247})

In [15]:
chat_history.add_ai_message(ai_response)
chat_history.add_user_message("Oh si, tengo otra pregunta. Será que piensan abrir una sucursal en Trujillo a futuro?")

ai_response = chain.invoke({
    "assistant_name": "Azzy",
    "company": "Shalom",
    "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
    "messages": chat_history.messages
})

print(ai_response)

content='Nuestro equipo está siempre evaluando nuevas oportunidades y posibles expansiones. Sin embargo, no tengo información específica sobre planes de abrir una sucursal en Trujillo en el futuro. Sería un proceso de investigación y planificación en la empresa, pero no puedo confirmar nada en este momento.' response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 280, 'total_tokens': 344, 'completion_time': 0.085333333, 'prompt_time': 0.06138251, 'queue_time': 0.0027741399999999944, 'total_time': 0.146715843}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None} id='run-5e7dc00c-130e-42e9-8c02-3f1824ab9bc3-0' usage_metadata={'input_tokens': 280, 'output_tokens': 64, 'total_tokens': 344}


In [16]:
chat_history.add_ai_message(ai_response)
chat_history.messages

[HumanMessage(content='Hola, ¿como estas?'),
 AIMessage(content='Estoy bien, gracias por preguntar. En que te puedo ayudar?'),
 HumanMessage(content='Me das tu nombre? Hacen envios a Trujillo?'),
 AIMessage(content='Soy Azzy, asistente virtual de Shalom. Solo hacemos envios a Piura, Lima y Chiclayo'),
 HumanMessage(content='Lástima, necesitaba un envio a Trujillo. Gracias por tu ayuda'),
 AIMessage(content='Lo siento, no podemos ayudarte con envíos a Trujillo. Si necesitas un servicio de courier a otras ciudades, tal vez podrías considerar otra empresa. Pero si estás planeando viajar a Piura, Lima o Chiclayo, ¡nosotros te podemos ayudar!', response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 182, 'total_tokens': 247, 'completion_time': 0.086666667, 'prompt_time': 0.049640882, 'queue_time': 0.0026035980000000056, 'total_time': 0.136307549}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id=

### Dándole mayor base de conocimiento al Chatbot con RAG (Retrieval Augmented Generation)

Muchas veces necesitaremos agregarle más contexto a la conversación para que el Chatbot pueda responder de manera más precisa. Sin embargo los LLM's suelen tener una capacidad máxima de tokens que pueden procesar.

Una empresa podría tener una base de datos de sus productos, servicios, o información de contacto, que podría ser usada para responder preguntas específicas de los usuarios, pero agregarlas directamente al modelo podría exceder la capacidad de tokens o generar altos costos de uso.

Para estos casos, LangChain nos permite agregar nuestra base de conocimientos a una base de datos vectorial (Vector Stores). Pueden ver la lista completa de librerías en https://python.langchain.com/v0.1/docs/integrations/vectorstores/

Asi mismo, nos permite recuperar los datos relevantes a usar en nuetro Prompt consultando la base de datos vectorial (Retriever). Pueden ver la lista completa de librerías en https://python.langchain.com/v0.1/docs/integrations/retrievers/

Para este ejemplo usaremos BeautifulSoup para la carga de documentos via web, y Chroma para la búsqueda y recuperación de información.

In [17]:
%pip install --upgrade --quiet langchain-chroma beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


La comunidad de LangChain es muy activa y siempre está creando nuevas librerías y herramientas para acelerar el desarrollo de nuestras aplicaciones. 

Para el ejemplo usaremos WebBaseLoder para cargar documentos via web usando BeautifulSoup.

Pueden ver la lista de Document Loaders disponibles en https://python.langchain.com/v0.1/docs/integrations/document_loaders/

In [19]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader([
    "https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html",
])

data = loader.load()
print(data)

[Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='\n\n\n\n\nPreguntas Frecuentes - Shalom\n\n\n\n\nBienvenido(a) a Preguntas Frecuentes\n\n\nResuelve tus dudas sobre nuestro servicio de envío.\nTus envíos\n¿Qué tipos de servicios ofrece Shalom?\nEn Shalom, nos especializamos en ofrecer servicios de envío de carga y encomiendas a nivel nacional, con más de 250 destinos terrestres y 15 destinos aéreos. Además, facilitamos el recojo y reparto de tus envíos directamente en tu domicilio. Gracias a nuestras alianzas estratégicas, también podrás realizar tus envíos internacionales de manera eficiente y confiable. Para conocer más visita nuestra página web en el siguiente enlace: https://shalom.com.pe\n¿Cuál es su cobertura?\nTenemos una amplia cobertura en todo el Perú, con más de 211 agencias a

Muchas veces los documentos puede ser muy extensos y dificiles de almacenar en el Vector Store. Además, es una buena práctica almacenar la información por partes semánticamente relacionadas para facilitar la recuperación de la información.

LangChain nos permite partir los documentos con diferentes técnicas, para el ejemplo usaremos RecursiveCharacterTextSplitter, el cual parte el texto en párrafos o grupos de caracteres según la longitud del texto.

Para ver una lista completa de los splitters disponibles pueden visitar https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/ 

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

all_splits

[Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='Preguntas Frecuentes - Shalom\n\n\n\n\nBienvenido(a) a Preguntas Frecuentes'),
 Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='Resuelve tus dudas sobre nuestro servicio de envío.\nTus envíos\n¿Qué tipos de servicios ofrece Shalom?'),
 Document(metadata={'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom', 'language': 'es'}, page_content='¿Qué tipos de servicios ofrece Shalom?\nEn Shalom, nos especi

Una vez tengamos nuestros documentos cargados y partidos, podemos almacenarlos en el Vector Store para poder recuperarlos en tiempo real.

Dado que los campos son almacenados como vectores, necesitaremos transformar los textos usando un modelo de embeddings.

Para este ejemplo usaremos el modelo de embeddings de Nomic el cual nos da 1 Millon de tokens gratis en su plan gratuito. Ver: https://atlas.nomic.ai/

Una lista completa de los modelos de embeddings disponibles en LangChain pueden ser encontrados en https://python.langchain.com/v0.1/docs/integrations/text_embedding/

In [21]:
%pip install -qU langchain-nomic

Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_nomic import NomicEmbeddings

embeddings = NomicEmbeddings(
    model="nomic-embed-text-v1.5"
)

embeddings.embed_query("Hola Python!!!")

[-0.029571533,
 0.047302246,
 -0.14819336,
 -0.030380249,
 0.039245605,
 0.008026123,
 -0.04864502,
 0.025314331,
 -0.014022827,
 -0.013069153,
 0.012557983,
 0.08618164,
 0.017486572,
 -0.025253296,
 0.024734497,
 -0.07623291,
 0.06311035,
 -0.013694763,
 -0.057037354,
 0.074645996,
 0.0006632805,
 -0.02645874,
 -0.046295166,
 -0.020614624,
 0.11254883,
 -0.030197144,
 -0.029388428,
 -0.0010070801,
 0.03918457,
 0.061828613,
 -0.0013465881,
 -0.036102295,
 -0.035736084,
 -0.04397583,
 -0.033477783,
 -0.017929077,
 0.09259033,
 -0.011734009,
 0.030715942,
 0.059631348,
 -0.061401367,
 0.007209778,
 -0.018936157,
 -0.016555786,
 0.029693604,
 -0.012565613,
 0.09399414,
 -0.03302002,
 -0.0045700073,
 0.009689331,
 -0.004436493,
 0.012367249,
 -0.05697632,
 -0.024169922,
 -0.00073337555,
 0.062927246,
 0.066223145,
 -0.01789856,
 0.044158936,
 0.018035889,
 0.05456543,
 0.061828613,
 -0.08514404,
 0.083862305,
 0.040008545,
 -0.043548584,
 -0.009277344,
 0.04736328,
 -0.058563232,
 0.0125

Almacenaremos los embeddings procesados en Chroma para poder recuperarlos más adelante.

Pueden revisar la documentación de Chroma en https://www.trychroma.com/ y la integración con LangChain en https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

In [23]:
from langchain_chroma import Chroma
from langchain_nomic import NomicEmbeddings

embedding = NomicEmbeddings(
    model="nomic-embed-text-v1.5"
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

vectorstore

El Retriever nos permitirá recuperar la información almacenada en nuestro Vector Store.

Por debajo trasnforma la pregunta del usuario en un vector y busca los vectores más cercanos en la base de datos vectorial.

In [24]:
retriever = vectorstore.as_retriever(k=5)
docs = retriever.invoke("Que medios de pago tienen?")

docs

[Document(metadata={'language': 'es', 'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom'}, page_content='¿Cuáles son los medios para realizar el pago de mi envío?\nShalom pone a su disposición nuestras 3 plataformas donde puede realizar el pago de sus envíos.'),
 Document(metadata={'language': 'es', 'source': 'https://gist.githubusercontent.com/willywg/25d405b96e8741979dee7382d3f84865/raw/648e29dcf9936333744249a76c75232c403704dd/envios_preguntas.html', 'title': 'Preguntas Frecuentes - Shalom'}, page_content='Página web: https://pagalo.shalom.pe\nApp Shalom: https://apple.co/3YNs2AK / https://cutt.ly/bXjJyz\nShalom Pro: https://pro.shalom.pe\n\nRecuerda que puedes realizar tus pagos con cualquiera de estos métodos:\n\nBilletera digital: Yape, Plin, Lukita, Tunki\nTarjeta\nEfectivo *En cualquiera de nuestras oficinas*\nPagos en Agencia, recuerd

### Chatbot con RAG

Finalmente, usaremos el modelo de RAG para recuperar la información relevante de la base de datos vectorial y agregarla al Prompt del Chatbot.

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain

load_dotenv(override=True) 

llm = ChatGroq(
    temperature=1.0,
    model="llama-3.1-8b-instant",
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente virtual. Tu nombre es {assistant_name} y trabajas en {company}. {description}. Usa las siguiente referencias como guía para tus respuestas: {context}"),
    MessagesPlaceholder(variable_name="messages"),
])

chat_history = ChatMessageHistory()

retriever = vectorstore.as_retriever(k=5)


In [26]:
def send_question(question):
    chat_history.add_user_message(question)
    docs = retriever.invoke(question)

    chain = create_stuff_documents_chain(llm, prompt)

    ai_response = chain.invoke({
        "assistant_name": "Azzy",
        "company": "Shalom",
        "description": "Shalom es unna empresa de courier. Solo hacen envios a Piura, Lima y Chiclayo",
        "messages": chat_history.messages,
        "context": docs
    })

    chat_history.add_ai_message(ai_response)
    return chat_history.messages

question = "Hola, ¿Que debo presentar para recoger un envio?"
send_question(question)

[HumanMessage(content='Hola, ¿Que debo presentar para recoger un envio?'),
 AIMessage(content='Hola, soy Azzy, tu asistente virtual de Shalom. Para recoger tu envío, debes presentar los siguientes documentos y realizar los siguientes pasos:\n\n* Si eres una persona natural, debes presentar tu DNI original y digitar la clave de seguridad.\n* Si eres una empresa, puedes seguir estos pasos:\n + Debes tener el contacto registrado en el ticket OS Shalom. En ese caso, debes presentar tu DNI original y digitar la clave de seguridad.\n + Si no tienes el contacto registrado, debes presentar una carta membretada debidamente firmada por el representante legal, copia de DNI, tu DNI original y digitar la clave de seguridad por tu parte.\n\nRecuerda verificar la información antes de presentarte para recoger tu envío. ¡Espero que todo salga bien!')]

In [27]:
send_question("Gracias ... y ¿que medios de pago tienen?")

[HumanMessage(content='Hola, ¿Que debo presentar para recoger un envio?'),
 AIMessage(content='Hola, soy Azzy, tu asistente virtual de Shalom. Para recoger tu envío, debes presentar los siguientes documentos y realizar los siguientes pasos:\n\n* Si eres una persona natural, debes presentar tu DNI original y digitar la clave de seguridad.\n* Si eres una empresa, puedes seguir estos pasos:\n + Debes tener el contacto registrado en el ticket OS Shalom. En ese caso, debes presentar tu DNI original y digitar la clave de seguridad.\n + Si no tienes el contacto registrado, debes presentar una carta membretada debidamente firmada por el representante legal, copia de DNI, tu DNI original y digitar la clave de seguridad por tu parte.\n\nRecuerda verificar la información antes de presentarte para recoger tu envío. ¡Espero que todo salga bien!'),
 HumanMessage(content='Gracias ... y ¿que medios de pago tienen?'),
 AIMessage(content='¡De nada! En Shalom, podemos realizar pagos de envíos de manera f

### Gracias

Espero que esta charla les haya sido de ayuda. Si tienen alguna pregunta, no duden en contactarme en LinkedIn: https://www.linkedin.com/in/willywg/

También pueden agregame en GitHub: https://github.com/willywg